<a href="https://colab.research.google.com/github/mitul01/FER/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/colab-20210303T131708Z-001/colab

/content/drive/MyDrive/colab-20210303T131708Z-001/colab


In [3]:
from keras.applications import ResNet50
# set matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import packages
from config import emotion_config as config
from pipeline.io import HDF5DatasetWriter
import numpy as np
import pandas as pd 
from keras.utils import to_categorical
from pipeline.preprocessing import ImageToArrayPreprocessor
from pipeline.io import HDF5DatasetGenerator
from pipeline.callbacks import EpochCheckpoint
from pipeline.callbacks import TrainingMonitor
from pipeline.io import HDF5DatasetGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.models import load_model
from keras import backend as K
import argparse
import os

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

In [9]:
trainAug = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)

valAug = ImageDataGenerator(rescale=1./255,)

iap = ImageToArrayPreprocessor()

trainGen_hdf5 = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
    aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES,)

valGen_hdf5 = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
    aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


#Resnet preloaded

In [10]:
model = ResNet50(classes = config.NUM_CLASSES,weights=None,input_shape=(48,48,1))

opt = Adam(lr = 1e-3)
model.compile(loss = "categorical_crossentropy", optimizer =opt,
    metrics = ["accuracy"])

figPath = os.path.sep.join([config.OUTPUT_PATH, "resnet50.png"])
jsonPath = os.path.sep.join([config.OUTPUT_PATH, "resnet50.json"])

callbacks = [
    EpochCheckpoint("checkpoints", every = 5, #startAt = args["start_epoch"]
                     ),TrainingMonitor(figPath, jsonPath = jsonPath) 
                     #startAt = args["start_epoch"]och"]                    
 ]

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

# train network

history=model.fit_generator(
    trainGen_hdf5.generator(),
    steps_per_epoch = trainGen_hdf5.numImages // config.BATCH_SIZE,
    validation_data = valGen_hdf5.generator(),
    validation_steps = valGen_hdf5.numImages // config.BATCH_SIZE,
    epochs = 150,
    max_queue_size = config.BATCH_SIZE * 2,
    callbacks = callbacks,
    verbose = 1
    
)
# close the dataset

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
224/224 [==============================] - 64s 115ms/step - loss: 2.3836 - accuracy: 0.2363 - val_loss: 1.8023 - val_accuracy: 0.2453
Epoch 2/150
224/224 [==============================] - 26s 112ms/step - loss: 1.7982 - accuracy: 0.2851 - val_loss: 7.0501 - val_accuracy: 0.2570
Epoch 3/150
224/224 [==============================] - 25s 112ms/step - loss: 1.7741 - accuracy: 0.3231 - val_loss: 6.6249 - val_accuracy: 0.3465
Epoch 4/150
224/224 [==============================] - 25s 113ms/step - loss: 1.6600 - accuracy: 0.3551 - val_loss: 1.7897 - val_accuracy: 0.2606
Epoch 5/150
224/224 [==============================] - 26s 115ms/step - loss: 1.5359 - accuracy: 0.3916 - val_loss: 1.4906 - val_accuracy: 0.3970
Epoch 6/150
224/224 [==============================] - 25s 113ms/step - loss: 1.4640 - accuracy: 0.4210 - val_loss: 10.7407 - val_accuracy: 0.3571
Epoch 7/150
224/224 [==============================] - 25s 113ms/step - loss: 1.4131 - accuracy: 0.4483 - val_loss: 1.8648 

In [11]:
testAug = ImageDataGenerator(rescale = 1 / 255.0)
iap = ImageToArrayPreprocessor()

testGen = HDF5DatasetGenerator(config.TEST_HDF5, config.BATCH_SIZE,
    aug = testAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [12]:
predictions = model.predict_generator(testGen.generator(),steps=testGen.numImages //64, max_queue_size=64*2)

(loss, acc) = model.evaluate_generator(
	testGen.generator(),
	steps=testGen.numImages // config.BATCH_SIZE,
	max_queue_size=config.BATCH_SIZE * 2)
print("[INFO] accuracy: {:.2f}".format(acc * 100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[INFO] accuracy: 61.44
